In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janeembednnmodelrishi/jane_embed_nn_model_rishi.pt
/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
import gc
import random
from   tqdm import tqdm
from   sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import psutil
import datatable as dt
from   collections import namedtuple

import os
import seaborn as sns
from   sklearn.utils import shuffle
import datetime
import time
from   sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef, roc_auc_score
from   sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from   torch.autograd import Variable
from   torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from   transformers import TrainingArguments
from   transformers import AdamW, BertConfig
from   transformers import get_linear_schedule_with_warmup

In [3]:
DEVICE = None

In [4]:
dtype = {
    'date'      : 'int64', 
    'weight'    : 'float64',
    'resp'      : 'float64',
    'ts_id'     : 'int64',  
    'feature_0' : 'float64'
}
for i in range (1, 130):
    k = 'feature_' + str (i)
    dtype[k] = 'float32'
    
X = pd.read_csv ('../input/jane-street-market-prediction/train.csv', dtype=dtype)
X.reset_index (drop=True, inplace=True)
y = X['resp']
X.drop (columns=['resp'], inplace=True)
opt_th = 0.25               # during prediction time, if resp > opt_th then resp = 1 else = 0
X.fillna (X.mean (), inplace=True)
f_columns = [c for c in X.columns if "feature" in c]
f_means   = np.mean (train[f_columns[1:]].values, axis=0)
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.1)
del X, y
gc.collect ()
X_train.head ()

,date,weight,resp_1,resp_2,resp_3,resp_4,feature_0,feature_1,feature_2,feature_3,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,18,3.773135,0.036276,0.052013,0.068639,0.078564,1.0,2.057005,0.875661,0.132545,...,6.455285,0.745585,2.865565,1.298025,5.703564,0.566450,2.424389,0.187537,1.508810,112788
1,363,0.279962,-0.015068,-0.021587,-0.023571,-0.009179,1.0,4.344903,7.735090,1.829789,...,-0.302524,2.052064,1.368755,1.911901,2.236057,3.573316,2.593936,2.850378,1.967827,1703355
2,435,1.083040,-0.000611,0.000684,0.000440,-0.005681,1.0,2.412920,1.732181,-0.880250,...,-1.368318,-0.860193,-1.889817,-0.308384,-1.458829,-0.480581,-1.114894,-0.999964,-1.823053,2044496
3,12,0.106268,-0.001211,-0.001211,0.003467,0.013918,1.0,5.211272,3.442069,2.550600,...,-1.406370,-0.017559,-1.086817,-0.071849,-1.720677,-0.076879,-1.228329,0.006170,-1.001456,83102
4,221,0.698037,0.000150,0.000627,0.000141,-0.003066,1.0,3.752843,3.163557,1.121520,...,-1.132460,4.183550,0.552824,3.026459,-0.117379,4.892067,0.387364,4.556887,0.429013,1050965


In [5]:
dtype = {
    'feature'  : 'str', 
    'tag_0'    : 'int8'
}
for i in range (1, 29):
    k = 'tag_' + str (i)
    dtype[k] = 'int8'

In [6]:
features_df = pd.read_csv ('../input/jane-street-market-prediction/features.csv', usecols=range(1,30), dtype=dtype)
N_FEATURES  = features_df.shape[0]  # the features.csv has 130 features (1st row) = no of features in train.csv (feature_0 to feature_129)
N_FEAT_TAGS = features_df.shape[1]  # the features.csv has 29 tags
features_df.head ()

,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,...,tag_19,tag_20,tag_21,tag_22,tag_23,tag_24,tag_25,tag_26,tag_27,tag_28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
del features_df
gc.collect ()

60

In [8]:
N_FEATURES, N_FEAT_TAGS

(130, 29)

# Custom Trainer

In [9]:
# Utilities for my custom trainer

def format_time (elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str (datetime.timedelta (seconds=elapsed_rounded))

def compute_regression_metrics (y_true, y_pred):
    """
    1d arrays of floats
    """
    
    mse = mean_squared_error (y_true, y_pred)
    mae = mean_absolute_error (y_true, y_pred)
    evs = explained_variance_score (y_true, y_pred)
    r2s = r2_score (y_true, y_pred)
    metrics = {
        'mse' : mse,
        'mae' : mae,
        'evs' : evs,
        'r2s' : r2s
    }
    return metrics

In [10]:
class MyTrainer:
    
    def __init__(self, model, args, train_dataset, eval_dataset, compute_regression_metrics=compute_regression_metrics):
        
        self.model           = model
        self.args            = args
        self.train_dataset   = train_dataset
        self.eval_dataset    = eval_dataset
        self.compute_regression_metrics = compute_regression_metrics
        self.isTrained       = False
        self.device          = self.get_device_type ()
        
        # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
        # I believe the 'W' stands for 'Weight Decay fix"
        self.optimizer = AdamW (model.parameters (),
                           lr  = args.learning_rate,
                           eps = args.adam_epsilon # args.adam_epsilon  - default is 1e-8 is “a very small number to prevent any division by zero"
        )

        # Number of training epochs. The BERT authors recommend between 2 and 4. 
        # We chose to run for 4, but we'll see later that this may be over-fitting the
        # training data.
        self.epochs = self.args.num_train_epochs
        if train_dataset:
            self.train_dataloader, self.validation_dataloader, self.lr_scheduler, self.num_training_steps = self.get_dataLoaders ()        
        return
    
    def get_device_type (self):
        
        # If there's a GPU available...
        if torch.cuda.is_available ():    

            # Tell PyTorch to use the GPU.    
            device = torch.device ("cuda")
            print('There are %d GPU(s) available.' % torch.cuda.device_count ())
            print('We will use the GPU:', torch.cuda.get_device_name (0))
        # If not...
        else:
            print('No GPU available, using the CPU instead.')
            device = torch.device ("cpu")
        
        global DEVICE
        DEVICE = device
        return device
    
    def get_dataLoaders (self):        
        
        # Create the DataLoaders for our training and validation sets.
        if isinstance (self.train_dataset, torch.utils.data.IterableDataset):
            train_sampler = None
        else:
            train_sampler = RandomSampler (self.train_dataset) # SequentialSampler (self.train_dataset)    # Better use RandomSampler
        train_dataloader  = DataLoader (
                    self.train_dataset,                                  # The training samples.
                    sampler     = train_sampler,                           
                    batch_size  = self.args.per_device_train_batch_size,
                    num_workers = 8    # TODO: uncomment this
        )
        # train_dataloader  = DataLoader (self.train_dataset, batch_size=self.args.per_device_train_batch_size)   # TODO comment this
        validation_dataloader = None
        if self.eval_dataset:
            
            # For validation the order doesn't matter, so we'll just read them sequentially.
            validation_dataloader = DataLoader (
                        self.eval_dataset,             # The validation/dev samples.
                        sampler     = SequentialSampler (self.eval_dataset),
                        batch_size  = self.args.per_device_eval_batch_size,
                        num_workers = 8    # TODO: uncomment this
            )
            # validation_dataloader = DataLoader (self.eval_dataset, batch_size=self.args.per_device_eval_batch_size)   # TODO comment this
            
        # Total number of training steps is [number of batches] x [number of epochs]. 
        # (Note that this is not the same as the number of training samples).
        num_training_steps = len (train_dataloader) * self.epochs

        # Create the learning rate scheduler.
        lr_scheduler = get_linear_schedule_with_warmup (self.optimizer, 
                                                        num_warmup_steps   = self.args.warmup_steps, # Default value in run_glue.py
                                                        num_training_steps = num_training_steps)
        return train_dataloader, validation_dataloader, lr_scheduler, num_training_steps
    
    
    def test_iterate_dataloader (self):
        
        for step, batch in enumerate (self.train_dataloader):
            print (step)
            print (batch)
            break
        return
    
    
    def train (self):
        
        # This training code is based on the `run_glue.py` script here:
        # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
        
        # Set the seed value all over the place to make this reproducible.
        seed_val = 42
        random.seed (seed_val)
        np.random.seed (seed_val)
        torch.manual_seed (seed_val)
        torch.cuda.manual_seed_all (seed_val)

        # We'll store a number of quantities such as training and validation loss, 
        # validation accuracy, and timings.
        training_stats = []
        # Measure the total training time for the whole run.
        total_t0 = time.time ()
        # inint min_val_loss to a large val, if after each epoch eval-loss < min_val_loss, then save the model
        min_val_loss   = 9999
        min_train_loss = 9999
        step = 0
        
        # For each epoch...
        for epoch_i in range (0, self.epochs):

            # ========================================
            #               Training
            # ========================================

            # Perform one full pass over the training set.

            print("")
            print('======== Epoch {:} / {:} ========'.format (epoch_i + 1, self.epochs))
            print('Training...')

            # Measure how long the training epoch takes.
            t0 = time.time ()

            # Reset the total loss for this epoch.
            total_train_loss = 0

            # Put the model into training mode. Don't be mislead--the call to 
            # `train` just changes the *mode*, it doesn't *perform* the training.
            # `dropout` and `batchnorm` layers behave differently during training
            # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
            self.model.train ()

            # For each batch of training data...
            for stp, batch in enumerate (self.train_dataloader):

                step += 1
                # Progress update every 40 batches.
                # print ('batch =', batch)
                if step % 50 == 0 and not step == 0:
                    
                    # Calculate elapsed time in minutes.
                    elapsed = format_time (time.time() - t0)
                    # Report progress.
                    print ('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len (self.train_dataloader), elapsed))
                if (self.args.max_steps > 0 and self.args.max_steps < step) or  \
                   (self.args.eval_steps> 0 and step % self.args.eval_steps==0 and step>0):
                    
                    avg_train_loss = total_train_loss / step
                    training_time = format_time (time.time () - t0)
                    if self.validation_dataloader:
                        
                        print ("Running Validation...")
                        avg_val_loss, avg_val_mae, avg_val_mse, avg_val_evs, avg_val_r2s, validation_time = self.evaluate ()
                        training_stats.append ({
                                'epoch'         : epoch_i + 1,
                                'training_loss' : avg_train_loss,
                                'eval_loss'     : avg_val_loss,
                                'eval_mae'      : avg_val_mae,
                                'eval_mse'      : avg_val_mse, 
                                'eval_evs'      : avg_val_evs,
                                'eval_r2s'      : avg_val_r2s,
                                'training_time' : training_time,
                                'eval_time'     : validation_time                   
                        })
                        # save this model if the eval loss decreases from the minimum so far
                        if avg_val_loss < min_val_loss: 

                            min_val_loss = avg_val_loss
                            torch.save (model.state_dict (), "jane_embed_nn_model.pt")     # TODO: uncomment this
                    if self.args.max_steps > 0 and self.args.max_steps < step :
                        print ("")
                        print ("Training complete!")
                        print ("Total training took {:} (h:mm:ss)".format (format_time (time.time ()-total_t0)))
                        self.isTrained = True
                        self.plot_train_stats_regression (training_stats)
                        return training_stats
                self.model.zero_grad ()        

                # Perform a forward pass (evaluate the model on this training batch).
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # It returns different numbers of parameters depending on what arguments
                # arge given and what flags are set. For our useage here, it returns
                # the loss (because we provided target) and the "logits"--the model
                # outputs prior to activation.
                # loss, logits = self.model (b_inputs, .., target=b_target)
                # print ('batch: \n', batch)
                # output    = self.model (date, weight, ts_id, features, target)                
                # Add batch to GPU
                for k in batch:
                    batch[k] = batch[k].to (self.device)
                output    = self.model (**batch)
                loss      = output.loss
                logits    = output.logits
                
                # Accumulate the training loss over all of the batches so that we can
                # calculate the average loss at the end. `loss` is a Tensor containing a
                # single value; the `.item()` function just returns the Python value 
                # from the tensor.
                total_train_loss += loss.item ()
                # Perform a backward pass to calculate the gradients.
                loss.backward ()

                # Clip the norm of the gradients to 1.0.
                # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                self.optimizer.step ()
                # Update the learning rate.
                self.lr_scheduler.step ()
            # At the end of each epoch measure stats and eval:
            # Calculate the average loss over all of the batches.
            avg_train_loss = total_train_loss / len (self.train_dataloader)
            # Measure how long this epoch took.
            training_time = format_time (time.time () - t0)            
            print ("  Average training loss: {0:.2f}".format (avg_train_loss))
            print ("  Training epcoh took: {:}".format (training_time))
            
            if self.validation_dataloader:
                
                print ("\n  Running Validation...")
                avg_val_loss, avg_val_mae, avg_val_mse, avg_val_evs, avg_val_r2s, validation_time = self.evaluate ()
                # Record all statistics from this epoch.
                training_stats.append ({
                        'epoch'         : epoch_i + 1,
                        'training_loss' : avg_train_loss,
                        'eval_loss'     : avg_val_loss,
                        'eval_mae'       : avg_val_mae,
                        'eval_mse'      : avg_val_mse, 
                        'eval_evs'      : avg_val_evs,
                        'eval_r2s'      : avg_val_r2s,
                        'training_time' : training_time,
                        'eval_time'     : validation_time                   
                })
                # save this epoch's model if the eval loss decreases from the minimum so far
                if avg_val_loss < min_val_loss:
                    
                    min_val_loss = avg_val_loss
                    torch.save (model.state_dict (), "jane_embed_nn_model.pt")     # TODO: uncomment this
            else:
                
                training_stats.append ({
                    'epoch'         : epoch_i + 1,
                    'training_loss' : avg_train_loss,
                    'training_time' : training_time,
                })
                if avg_train_loss < min_train_loss: 
                    
                    min_train_loss = avg_train_loss
                    torch.save (model.state_dict (), "jane_embed_nn_model.pt")     # TODO: uncomment this
        print ("")
        print ("Training complete!")
        print ("Total training took {:} (h:mm:ss)".format (format_time (time.time ()-total_t0)))
        self.isTrained = True
        self.plot_train_stats_regression (training_stats)
        return training_stats
    
    def evaluate (self):
        
        t0 = time.time ()
        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        self.model.eval ()

        # Tracking variables 
        total_eval_mae  = 0
        total_eval_mse  = 0
        total_eval_evs  = 0
        total_eval_r2s  = 0
        total_eval_loss = 0
        nb_eval_steps   = 0

        # Evaluate data for one epoch
        for batch in self.validation_dataloader:
                        
            # Add batch to GPU
            for k in batch:
                batch[k] = batch[k].to (self.device)
            with torch.no_grad ():
                
                # Forward pass, calculate logit predictions.
                output    = self.model (**batch)
                loss      = output.loss
                logits    = output.logits

            # Accumulate the validation loss.
            total_eval_loss += loss.item ()
            # Move logits and target to CPU
            logits    = logits.detach ().cpu ().numpy ()
            target    = batch['target']
            label_ids = target.numpy ()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            metrics = self.compute_regression_metrics (label_ids, logits)
            total_eval_mae += metrics['mae']
            total_eval_mse += metrics['mse']
            total_eval_evs += metrics['evs']
            total_eval_r2s += metrics['r2s']

        # Report the final accuracy for this validation run.
        avg_val_mae = total_eval_mae / len (self.validation_dataloader)
        print ("  MAE: {0:.3f}".format (avg_val_mae))
        avg_val_mse = total_eval_mse / len (self.validation_dataloader)
        print ("  MSE: {0:.3f}".format (avg_val_mse))
        avg_val_evs = total_eval_evs / len (self.validation_dataloader)
        print ("  EVS: {0:.3f}".format (avg_val_evs))
        avg_val_r2s = total_eval_r2s / len (self.validation_dataloader)
        print ("  R2 : {0:.3f}".format (avg_val_r2s))
        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len (self.validation_dataloader)
        # Measure how long the validation run took.
        validation_time = format_time (time.time () - t0)
        print ("  Validation Loss: {0:.2f}".format (avg_val_loss))
        print ("  Validation took: {:}".format (validation_time))            
        return avg_val_loss, avg_val_mae, avg_val_mse, avg_val_evs, avg_val_r2s, validation_time
    
    def plot_train_stats_regression (self, training_stats):
        """
        Draw Classification Report curve
        """
        
        mae = mse = evs = r2s = ev_losses = tr_losses = -1
        epochs = training_stats[-1]['epoch']
        if 'eval_mae' in training_stats[0]:
            mae       = [e['eval_mae'] for e in training_stats]
            sns.lineplot (x=np.arange(1, epochs + 1), y=mae,       label='val_mae')
        if 'eval_mse' in training_stats[0]:
            mse = [e['eval_mse'] for e in training_stats]
            sns.lineplot (x=np.arange(1, epochs + 1), y=mse, label='val_mse')
        if 'eval_evs' in training_stats[0]:
            evs  = [e['eval_evs'] for e in training_stats]
            sns.lineplot (x=np.arange(1, epochs + 1), y=evs,  label='val_evs') 
        if 'eval_r2s' in training_stats[0]:
            r2s = [e['eval_r2s'] for e in training_stats]
            sns.lineplot (x=np.arange(1, epochs + 1), y=r2s, label='val_r2s')
        if 'eval_loss' in training_stats[0]:
            ev_losses  = [e['eval_loss'] for e in training_stats]
            sns.lineplot (x=np.arange(1, epochs + 1), y=ev_losses,  label='ev_losses')
        if 'training_loss'  in training_stats[0]:
            tr_losses  = [e['training_loss'] for e in training_stats]
            sns.lineplot (x=np.arange(1, epochs + 1), y=tr_losses,  label='tr_losses')
            
        plt.show ()
        print ('mae       :', mae)
        print ('mse       :', mse)
        print ('evs       :', evs)
        print ('r2s       :', r2s)
        print ('ev_losses :', ev_losses)
        print ('tr_losses :', tr_losses)
        return
    
    
    def getTrainedModel (self):
        
        if self.isTrained:
            return self.model
        return None
    
    
    def predict (self, prediction_dataset, isRemovetarget=True):
        """
        return: pred_logits, true_target, metrics (if true 'target' are input in the prediction_dataset)
        """     
        
        prediction_sampler    = SequentialSampler (prediction_dataset)
        prediction_dataloader = DataLoader (prediction_dataset, sampler=prediction_sampler, batch_size=self.args.per_device_eval_batch_size, num_workers=8)
        print ('Predicting target for {:,} records'.format (len (prediction_dataset)))
        
        # Put model in evaluation mode
        self.model.eval ()

        # Tracking variables 
        predictions = []
        # true_target = []
        
        # Predict 
        for step, batch in enumerate (prediction_dataloader):
            
            # Unpack the inputs from our dataloader
            # b_input_ids, b_input_mask, b_segment_ids = batch

            # Telling the model not to compute or store gradients, saving memory and 
            # speeding up prediction
            with torch.no_grad ():
                # Forward pass, calculate logit predictions
                if 'target' in batch and isRemovetarget:
                    batch.pop ('target')
                # Add batch to GPU
                for k in batch:
                    batch[k] = batch[k].to (self.device)
                outputs = model (**batch)
            logits = outputs.logits

            # Move logits and target to CPU
            logits = logits.detach ().cpu ().numpy ()
            # label_ids = b_target.to ('cpu').numpy ()

            # Store predictions and true target
            predictions.append (logits)
            # true_target.append (label_ids)
            print ('Done predictions for ', len(predictions), '/', len(prediction_dataloader), 'batches')
        print ('Done prediction')
        
        # Combine the results across all batches to get the predicted logits
        pred_logits = np.concatenate (predictions, axis=0)
        # For each sample, pick the label (0,1,2) with the highest score.
        # pred_target = np.argmax (pred_logits, axis=1).flatten()
        # returns the predicted logits
        return pred_logits, None, None

# Create Datasets

In [11]:
# create a custom map type dataset
class JaneDataset (torch.utils.data.Dataset):
    
    def __init__(self, df, target=None):
        
        self.df     = df
        self.target = target 
        return
    
    def __len__(self):
        return len (self.df)
    
    def __getitem__(self, idx):
        
        if torch.is_tensor (idx):
            idx = idx.tolist ()
        
        date      = self.df['date'][idx]
        weight    = self.df['weight'][idx]
        # ts_id   = self.df['ts_id'][idx]
        sample    = None
        f_columns = ['feature_' + str(i) for i in range (N_FEATURES)]
        if self.target is not None:
            features  = np.array (self.df.iloc[idx, 7:7+N_FEATURES])
            sample = {'date': date, 'weight': weight, 'features': features, 'target': self.target[idx]}
        else:
            features  = np.array (self.df.loc[idx, f_columns])
            sample = {'date': date, 'weight': weight, 'features': features}
        return sample

In [12]:
train_dataset = JaneDataset (X_train, y_train)
eval_dataset  = JaneDataset (X_test,  y_test)
train_dataset[4]                                    # output    = self.model (date, weight, ts_id, features, target)

{'date': 221,
 'weight': 0.6980374615455953,
 'features': array([ 3.75284290e+00,  3.16355729e+00,  1.12151992e+00,  7.98064470e-01,
         7.58217573e-02,  4.23717871e-02, -2.84580559e-01, -3.70991439e-01,
         4.33722305e+00,  2.80456519e+00,  1.23256278e+00,  1.56172097e+00,
         2.66169739e+00,  1.91029882e+00,  3.06427264e+00,  3.19043851e+00,
         1.03649354e+00,  1.10475636e+00,  3.79440618e+00,  3.91434026e+00,
         2.43178344e+00,  1.39430583e+00,  2.96089911e+00,  3.27297020e+00,
         3.99362612e+00,  4.02758503e+00, -3.84995669e-01, -5.35521984e-01,
        -1.75935590e+00, -2.13429379e+00, -7.79497623e-01, -1.26306725e+00,
        -1.40747547e+00, -1.28880656e+00, -2.64053607e+00, -2.15611982e+00,
         2.51348686e+00,  2.41272259e+00, -1.33672625e-01, -2.03591168e-01,
        -8.53877962e-01,  4.11715412e+00,  1.07402062e+00, -2.62250638e+00,
        -1.58683753e+00,  3.16836178e-01,  1.42270362e+00,  1.10198820e+00,
         7.60594487e-01, -6.076

# Model

In [13]:
class FFN (nn.Module):
    
    def __init__(self, state_size=200):
        
        super (FFN, self).__init__()
        self.state_size = state_size
        self.lr1        = nn.Linear (state_size, state_size)
        self.relu       = nn.ReLU ()
        self.lr2        = nn.Linear (state_size, state_size)
        self.dropout    = nn.Dropout (0.2)
    
    def forward (self, x):
        
        x = self.lr1 (x)
        x = self.relu (x)
        x = self.lr2 (x)
        x = self.relu (x)
        return self.dropout (x)

In [14]:
class Regression_model (nn.Module):
    
    def __init__(self, embed_dim=N_FEAT_TAGS//4, csv_file='../input/jane-street-market-prediction/features.csv'):
        
        super (Regression_model, self).__init__()
        
        # store the features to tags mapping as a datframe tdf, feature_i mapping is in tdf[i, :]
        dtype = {'tag_0' : 'int8'}
        for i in range (1, 29):
            k = 'tag_' + str (i)
            dtype[k] = 'int8'
        self.features_tag_matrix = torch.tensor (pd.read_csv (csv_file, usecols=range (1,N_FEAT_TAGS+1), dtype=dtype).to_numpy ())
        
        # embeddings for the tags. Each feature is taken a an embedding which is an avg. of its' tag embeddings
        self.embed_dim     = embed_dim
        self.tag_embedding = nn.Embedding (N_FEAT_TAGS+1, embed_dim) # create a special tag if not known tag for any feature
        self.tag_weights   = nn.Linear (N_FEAT_TAGS, 1)
        
        self.dropout      = nn.Dropout (0.2)
        self.layer_normal = nn.LayerNorm (embed_dim) 
        self.ffn          = FFN (embed_dim)
        self.outDense     = nn.Linear (embed_dim, 1)
        self.criterion    = nn.L1Loss ()
        return
    
    def features2emb (self):
        """
        idx : int feature index 0 to N_FEATURES-1 (129)
        """
        
        all_tag_idxs = torch.LongTensor (np.arange (N_FEAT_TAGS)).to (DEVICE)              # (29,)
        tag_bools    = self.features_tag_matrix                                # (130, 29)
        # print ('tag_bools.shape =', tag_bools.size())
        f_emb        = self.tag_embedding (all_tag_idxs).repeat (130, 1, 1)    #;print ('1. f_emb =', f_emb) # (29, 7) * (130, 1, 1) = (130, 29, 7)
        # print ('f_emb.shape =', f_emb.size())
        f_emb        = f_emb * tag_bools[:, :, None]                           #;print ('2. f_emb =', f_emb) # (130, 29, 7) * (130, 29, 1) = (130, 29, 7)
        # print ('f_emb.shape =', f_emb.size())
        
        # Take avg. of all the present tag's embeddings to get the embedding for a feature
        # s = torch.sum (tag_bools, dim=1)
        # print ('s.shape =', s.shape)
        # print ('s =', s)
        # f_emb = torch.sum (f_emb, dim=-2) / s[:, None]                       # (130, 7)
        # print ('f_emb.shape =', f_emb.size())        
        # print ('f_emb.shape =', f_emb.shape)
        
        # take a linear combination of the present tag's embeddings
        f_emb = f_emb.permute (0, 2, 1)                                        # (130, 7, 29)
        f_emb = self.tag_weights (f_emb)                      #;print ('3. f_emb =', f_emb)                 # (130, 7, 1)
        f_emb = torch.squeeze (f_emb, dim=-1)                 #;print ('4. f_emb =', f_emb)                 # (130, 7)
        return f_emb
    
    def forward (self, date, weight, features, target=None):
        """
        when you call `model (x ,y, z, ...)` then this method is invoked
        """
        
        f_emb    = self.features2emb ()                                #;print ('5. f_emb =', f_emb); print ('6. features =', features) # (130, 7)
        # print ('features.shape =', features.shape, 'f_emb.shape =', f_emb.shape)
        features = torch.matmul (features, f_emb)                      #;print ('7. features =', features) # (1, 130) * (130, 7) = (1, 7)
        # print ('features.shape =', features.shape)
        
        x          = self.ffn (features)                               #;print ('8. x.shape = ', x.shape, 'x =', x) 
        x          = self.layer_normal (x + features)                  #;print ('9. x.shape = ', x.shape, 'x =', x) 
        out_logits = self.outDense (x)                                 #;print ('10. out_logits.shape = ', out_logits.shape, 'out_logits =', out_logits)
        out_logits = torch.squeeze (out_logits, dim=-1)
        if target is None:
            
            # return a named tuple
            Logits     = namedtuple ('Logits',['logits'])
            out_logits = Logits (out_logits)
            return out_logits                                          # you can access the value as out_logits.logits        
        
        # calculate loss, this is used for back propagation later
        batchLoss  = self.criterion (out_logits, target)
        
        # return a named tuple
        Loss_Logits = namedtuple ('Loss_Logits',['loss','logits'])
        loss_logits = Loss_Logits (batchLoss, out_logits)             # you can access the value as out_logits.loss, out_logits.logits
        #print ('11. out_logits =', out_logits)
        return loss_logits

In [15]:
model = Regression_model ()
try:
    model.load_state_dict (torch.load ("../input/janeembednnmodelrishi/jane_embed_nn_model_rishi.pt"))
except:
    model.load_state_dict (torch.load ("../input/janeembednnmodelrishi/jane_embed_nn_model_rishi.pt", map_location='cpu'))
model = model.double ()

In [16]:
training_args = TrainingArguments (

    output_dir      = './results',     # output directory
    num_train_epochs= 5,               # for training from scratch
    warmup_steps    = 50,              # for lr scheduling
    eval_steps      = 50,              # Number of update steps between two evaluations, if <=0 then eval at end of each epoch
    max_steps       = 0,               # If set to a positive number, the total number of training steps to perform. Overrides num_train_epochs
    learning_rate   = 1e-2,            # Actually = 1e-2 for training from scratch, without using the jane_embed_nn_model.pt
    # adam_epsilon  = 1e-8             # - default is 1e-8 is “a very small number to prevent any division by zero"
    per_device_train_batch_size= 50240, # batch size per device during training
    per_device_eval_batch_size = 50240, # batch size for evaluation
)
trainer = MyTrainer (
    
    model         = model,           # the instantiated 🤗 Transformers model to be trained
    args          = training_args,   # training arguments, defined above
    train_dataset = train_dataset,   # training dataset
    eval_dataset  = eval_dataset,    # evaluation dataset
)

No GPU available, using the CPU instead.


In [17]:
# trainer.test_iterate_dataloader ()

In [18]:
# uncomment this to train from scratch
# trainer.train ()

In [19]:
training_args = TrainingArguments (

    output_dir      = './results',     # output directory
    num_train_epochs= 2,               # for training from scratch
    warmup_steps    = 50,              # for lr scheduling
    eval_steps      = 50,              # Number of update steps between two evaluations, if <=0 then eval at end of each epoch
    max_steps       = 0,               # If set to a positive number, the total number of training steps to perform. Overrides num_train_epochs
    learning_rate   = 5e-5,            # Actually = 1e-2 for training from scratch, without using the jane_embed_nn_model.pt
    # adam_epsilon  = 1e-8             # - default is 1e-8 is “a very small number to prevent any division by zero"
    per_device_train_batch_size= 50240, # batch size per device during training
    per_device_eval_batch_size = 50240, # batch size for evaluation
)
trainer = MyTrainer (
    
    model         = model,           # the instantiated 🤗 Transformers model to be trained
    args          = training_args,   # training arguments, defined above
    train_dataset = eval_dataset,    # training dataset
    eval_dataset  = None,            # evaluation dataset
)

No GPU available, using the CPU instead.


In [20]:
# uncomment this to train from scratch
# trainer.train ()

In [21]:
# uncomment this to train from scratch
# Load the best model so far
# del model
# gc.collect ()
# model = Regression_model ()
# try:
#     model.load_state_dict (torch.load ("jane_embed_nn_model.pt"))
# except:
#     model.load_state_dict (torch.load ("jane_embed_nn_model.pt", map_location='cpu'))
# model = model.double ()

# Test

In [22]:
import janestreet
env      = janestreet.make_env() # initialize the environment
env_iter = env.iter_test() # an iterator which loops over the test set

In [23]:
# this is useless but just reusing this
training_args = TrainingArguments (

    output_dir      = './results',     # output directory
    num_train_epochs= 10,              # for training from scratch
    warmup_steps    = 500,             # for lr scheduling
    eval_steps      = 0,               # Number of update steps between two evaluations, if <=0 then eval at end of each epoch
    max_steps       = 2,               # If set to a positive number, the total number of training steps to perform. Overrides num_train_epochs
    learning_rate   = 1e-2,            # Actually = 1e-2 for training from scratch, without using the jane_embed_nn_model.pt
    # adam_epsilon  = 1e-8             # - default is 1e-8 is “a very small number to prevent any division by zero"
    per_device_train_batch_size= 5024, # batch size per device during training
    per_device_eval_batch_size = 5024, # batch size for evaluation
)

trainer = MyTrainer (

    model         = model,           # the instantiated 🤗 Transformers model to be trained
    args          = training_args,   # training arguments, defined above
    train_dataset = None,            # training dataset
    eval_dataset  = None,            # evaluation dataset
)

No GPU available, using the CPU instead.


# For submission using the Trainer class

for (test_df, sample_prediction_df) in iter_test:
    
    test_df.reset_index (drop=True, inplace=True)
    test_df.fillna (0, inplace = True)
    test_dataset = JaneDataset (test_df)    
    y, _, _      = trainer.predict (test_dataset)
    sample_prediction_df.action = y
    env.predict (sample_prediction_df)

# For direct submission, without using the Trainer class

In [ ]:
opt_th = 0.0                           # during prediction time, if resp > opt_th then resp = 1 else = 0
for test_df, pred_df in env_iter:
    if test_df["weight"].item () > 0:
        
        test_df.reset_index (drop=True, inplace=True)
        test_df.fillna (0, inplace = True)
        
        # x_tt = test_df.loc[:, f_columns].values
        # if np.isnan (x_tt[:, 1:].sum ()):
        #     x_tt[:, 1:] = np.nan_to_num (x_tt[:, 1:]) + np.isnan (x_tt[:, 1:]) * f_mean        
        # replace feature values
        # test_df.drop (columns=f_columns, inplace = True)
        # test_df[f_columns] = x_tt
        
        test_dataset = JaneDataset (test_df)
        pred = []
        for i in range (len (test_dataset)):
            
            batch = test_dataset[i]
            # Add batch to GPU
            for k in batch:
                batch[k] = torch.tensor (batch[k]).to (DEVICE)
            pred.append (model (**batch).logits.item ())
        
        pred = np.array (pred)
        pred_df.action = np.where (pred >= opt_th, 1, 0).astype (int)
    else:
        pred_df.action = 0
        
    # print (pred_df)
    # print ("--------------")
    env.predict (pred_df)

In [25]:
print ('Done !')

Done !
